# 재구매 추천 알림을 위한 평균 기간 계산 로직

In [2]:
import pandas as pd
import numpy as np

user = pd.read_csv('./사용자데이터.csv', encoding='cp949')
order = pd.read_csv('./구매데이터.csv', encoding='cp949')
item = pd.read_csv('./상품데이터.csv', encoding='cp949')
user

,USER_ID,USER_PW,NAME,GENDER,AGE,PURPLE_BOX,MEANING_SCORE
0,kurlyholic,kurlyholic,kurlyholic,Female,20,Y,0.0
1,DxsDtzxrz,DxsDtzxrz,Matyukon,Female,40,N,0.0
2,Msz1s6T,Msz1s6T,Veck,Female,50,N,0.0
3,tfSvZKK,tfSvZKK,Rydeard,Female,50,N,918.0
4,WpENmxH,WpENmxH,Tidey,Female,40,N,1030.0
...,...,...,...,...,...,...,...
1996,LtWPTXWH9,LtWPTXWH9,Cathee,Female,50,N,0.0
1997,GJ8E56jR,GJ8E56jR,Buckston,Female,30,N,0.0
1998,w7dqaeYOi,w7dqaeYOi,Treece,Female,40,Y,0.0
1999,GRepdjnCIzS,GRepdjnCIzS,Durkin,Male,50,N,555.0


In [3]:
from datetime import datetime
import json

dataset = pd.merge(order, item, on=['ITEM_ID', 'ITEM_NAME'], how='left')
dataset

def strptime(x):
    if pd.isna(x):
        return 0
    x = x.split('|')
    a = datetime.strptime(x[0], '%Y-%m-%d %H:%M')
    b = datetime.strptime(x[1], '%Y-%m-%d %H:%M')
    return (a-b).days

dataset = dataset.sort_values(by=['USER_ID', 'ITEM_ID', 'BUY_DT'])
dataset['TOTAL_ID'] = dataset['USER_ID'].astype(str)+dataset['ITEM_ID'].astype(str)
dataset['order_cnt'] = [1 for _ in range(len(dataset))]
cumsum_df = dataset.groupby(['TOTAL_ID', 'USER_ID', 'ITEM_ID', 'BUY_DT', 'CATEGORY'])['order_cnt'].sum().groupby(level=0).cumsum().reset_index()
cumsum_df = cumsum_df.sort_values(['TOTAL_ID', 'order_cnt'])
cumsum_df['date_diff'] = cumsum_df['BUY_DT'] +'|'+ cumsum_df['BUY_DT'].shift()
cumsum_df['date_diff'] = cumsum_df['date_diff'].apply(strptime)
cumsum_df['date_diff'] = [np.nan if x['order_cnt'] == 1 else x['date_diff'] for idx, x in cumsum_df.iterrows()]
cumsum_df

,TOTAL_ID,USER_ID,ITEM_ID,BUY_DT,CATEGORY,order_cnt,date_diff
0,00h3IkoZ5J1136247046,00h3IkoZ5J,1136247046,2022-08-27 16:43,헬스기구,1,NaN
1,00h3IkoZ5J1137144555,00h3IkoZ5J,1137144555,2022-05-05 7:14,스낵과자,1,NaN
2,00h3IkoZ5J1163003241,00h3IkoZ5J,1163003241,2022-01-20 12:05,스낵과자,1,NaN
3,00h3IkoZ5J1163003241,00h3IkoZ5J,1163003241,2022-04-30 1:14,스낵과자,2,99.0
4,00h3IkoZ5J1334683997,00h3IkoZ5J,1334683997,2022-08-17 4:48,회화용구,1,NaN
...,...,...,...,...,...,...,...
33002,zzhhoF7pq36861276479,zzhhoF7pq3,6861276479,2022-08-25 6:10,현미,1,NaN
33003,zzhhoF7pq383627809788,zzhhoF7pq3,83627809788,2022-04-18 8:27,생수,1,NaN
33004,zzhhoF7pq383749289789,zzhhoF7pq3,83749289789,2022-05-14 13:46,마른멸치,1,NaN
33005,zzhhoF7pq3864710605,zzhhoF7pq3,864710605,2022-05-17 10:56,아동복,1,NaN


In [4]:
rebuy_df = cumsum_df[pd.notnull(cumsum_df['date_diff'])]
rebuy_df = rebuy_df.groupby(['CATEGORY'])['date_diff'].mean().reset_index()
rebuy_df

,CATEGORY,date_diff
0,가정용비닐용품,42.900000
1,간장,50.172414
2,고구마,54.228571
3,고등어,74.275000
4,고사리,59.888889
...,...,...
122,혼식곡,57.023256
123,혼합음료,57.884615
124,혼합조미료,46.142857
125,화장지,83.666667


In [5]:
rebuy_df.to_csv('./rebuy.csv', encoding='cp949', index=False)

In [6]:
last_order = order.groupby(['USER_ID', 'ITEM_ID'])['BUY_DT'].max().reset_index()
now  = datetime.now()
diff = now - pd.to_datetime(last_order['BUY_DT'], format="%Y-%m-%d %H:%M:%S")
last_order['RE_ORDER_BEFORE'] = diff.dt.days
last_order = pd.merge(last_order, item, on=['ITEM_ID'], how='left')
last_order = pd.merge(last_order, rebuy_df, on=['CATEGORY'], how='left')
last_order

,USER_ID,ITEM_ID,BUY_DT,RE_ORDER_BEFORE,CATEGORY,ITEM_NAME,SALE_PRICE,REDUCED_PRICE,BENEFIT_PRICE,DISCOUNT_RATE,IS_FOOD,ITEM_FEATURE1,ITEM_FEATURE2,MEANING_SCORE,PRICE_DT,KURLY_HCT,KURLY_MCT,date_diff
0,00h3IkoZ5J,559022629,2022-08-10 22:57,13,아동복,데이즈[겨울왕국]여아내복,"10,000","10,000",0,-,비식품,NaN,NaN,NaN,2022-07-01,베이비_키즈_완구,아동패션,53.526316
1,00h3IkoZ5J,1136247046,2022-08-27 16:43,-4,헬스기구,스타 역기봉 역도 바벨 헬스 운동 홈트레이닝 5.1kg,"30,700","30,700",0,-,비식품,NaN,NaN,NaN,2022-07-01,가전제품,헬스기구,40.692308
2,00h3IkoZ5J,1137144555,2022-05-05 7:14,111,스낵과자,오리온 후레쉬베리 복숭아 336g(12개입)x3,"16,900","16,900",0,-,식품,NaN,NaN,NaN,2022-07-01,간식_과자_떡,과자_스낵_쿠키,51.419355
3,00h3IkoZ5J,1163003241,2022-04-30 1:14,116,스낵과자,키와 안데안 포테이토 칩 믹스 50g,"3,000","3,000",0,-,식품,NaN,컬리온리,NaN,2022-07-01,간식_과자_떡,과자_스낵_쿠키,51.419355
4,00h3IkoZ5J,1334683997,2022-08-17 4:48,7,회화용구,(9793010) 다이아몬드 색연필 노랑색 1갑,"2,640","2,640",0,-,비식품,NaN,NaN,NaN,2022-07-01,생활용품_리빙_캠핑,생활잡화_문구,56.458333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29992,zzhhoF7pq3,32433127809,2022-08-17 3:57,7,냉동식품,CJ비비고왕교자455G*2EAx1개,"8,480","8,480",0,-,식품,NaN,NaN,NaN,2022-07-01,간식_과자_떡,아이스크림,52.503876
29993,zzhhoF7pq3,32937736543,2022-08-22 9:57,2,참기름,백설 진한참기름450ml,"7,880","7,880",0,-,식품,NaN,NaN,NaN,2022-07-01,면_양념_오일,식용유_참기름_오일,49.882353
29994,zzhhoF7pq3,33210473648,2022-08-14 10:51,10,참기름,CJ백설 진한참기름320ml,"6,310","6,310",0,-,식품,NaN,NaN,NaN,2022-07-01,면_양념_오일,식용유_참기름_오일,49.882353
29995,zzhhoF7pq3,83627809788,2022-04-18 8:27,128,생수,제주 삼다수 무라벨 라벨 330ml 500ml 2l 가정 사무실 학교 그린 생수 정기배달,"4,500","4,500",0,-,식품,친환경,NaN,225.0,2022-07-01,생수_음료_우유_커피,생수_탄산수,61.714286


In [7]:
rebuy_alarm = last_order[last_order['RE_ORDER_BEFORE'] > last_order['date_diff']]
rebuy_alarm = rebuy_alarm.groupby(['USER_ID'])['ITEM_ID'].apply(list).reset_index()

In [8]:
rebuy_dic = {}
for idx, row in rebuy_alarm.iterrows():
    rebuy_dic[row['USER_ID']] = row['ITEM_ID']

json.dumps(rebuy_dic)

'{"00h3IkoZ5J": [1137144555, 1163003241, 1335525769, 1340195272, 24653936680, 31354097996, 31963091416, 33156474065, 83358587789], "06g7tiKVUvE": [571990331, 799888698, 1197206062, 1244554148, 2048560134, 11058655118, 23664442345, 24292507125, 31026796825, 32415389213, 32962576447, 82815037633], "07R28iXhF": [671336118, 1308004474, 1624143059, 29713136212, 32766169696], "08QhrW": [974897067, 1211561758, 1519659789, 1621708598, 1973709044, 15598275523, 22433339325, 25244435400, 30321230933, 31603508802, 32591314752], "0AR242": [693203071, 1094129141, 1342602472, 24102764012, 29714485470, 29761397848, 31506122615, 31912221372, 80097449873], "0AZrC7FP6": [227178796, 1045293928, 1197113874, 24609723731, 25867784093, 30330509648, 31262732137, 31993644971, 33193768650, 82074706357, 82748302585, 84171160658], "0DYYtGkKYvnw": [23841810988, 26122541158, 31458818043, 32041918001], "0E5DtX4umt7": [188608972, 280001355, 916625735, 1271302406], "0HRWCgimqavl": [1584037017, 1768396270, 23280323718, 

# 가치소비 상품 추천 로직

In [9]:
rate = 0.2
user_num = int(len(user)*rate)

meaning_top_user = user.sort_values(by=['MEANING_SCORE'], ascending=False)
meaning_top_user = meaning_top_user[:user_num]

In [10]:
meaning_order = pd.merge(meaning_top_user, order, on=['USER_ID'], how='left')
meaning_order = pd.merge(meaning_order, item, on=['ITEM_ID', 'ITEM_NAME'], how='left')
meaning_order = meaning_order[pd.notnull(meaning_order['ITEM_FEATURE1'])]
item_list = meaning_order.groupby(['ITEM_ID'])['ITEM_NAME'].count().reset_index().sort_values(by=['ITEM_NAME'], ascending=False)
list(item_list['ITEM_ID'])[:20]

[32937507447,
 29789635621,
 28954847674,
 32843605785,
 33186477647,
 22602596724,
 82767467741,
 30952888706,
 32962104874,
 82888782218,
 22023812540,
 29424148848,
 12824986818,
 31276635379,
 1320849702,
 25318828367,
 26005611130,
 84396183047,
 33208888893,
 32919556101]